In [1]:
import sys, os
sys.path.append('./..')
sys.path.append(["/Users/taylordupuy/Documents/web-development/dev/flaskpage","/Users/taylordupuy/Documents/web-development/dev/flaskpage/project"])

In [2]:
#without adding data to the database queries will not work.
os.remove('/Users/taylordupuy/Documents/web-development/dev/flaskpage/project/db.sqlite')

In [3]:
#This command needs to be run from the auth environment or it won't run.
#from project import db, create_app
#db.create_all(app=create_app())

""" 
from project import db, create_app

app=create_app()
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite'
with app.app_context():
    db.create_all()
"""

"""
with app.app_context():
    users=db.session.query(User).all()
"""

"""
#import models fails
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from models import User, Submission, Problem, Course

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite'
dbb = SQLAlchemy(app)
dbb.init_app(app)
"""

"\n#import models fails\nfrom flask import Flask\nfrom flask_sqlalchemy import SQLAlchemy\nfrom models import User, Submission, Problem, Course\n\napp = Flask(__name__)\napp.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite'\ndbb = SQLAlchemy(app)\ndbb.init_app(app)\n"

In [4]:
############################
from werkzeug.security import generate_password_hash, check_password_hash

sys.path.append('/users/taylordupuy/Documents/web-development/dev/excel_tools')
sys.path.append('/users/taylordupuy/Documents/web-development/dev/email_tools')

from table_editor import SheetObject

R=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','roster')
S=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','submissions')
A=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','assignments')
P=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster-test.xlsx','assignments')

from database_declarative import User, Submission, Problem, Course, Base
#from models import db
#from models import User, Submission, Problem, Course
from sqlalchemy import create_engine
import numpy as np
from email_functions import *
from sqlalchemy import or_, and_
from sqlalchemy.orm import relationship

engine = create_engine('sqlite:///db.sqlite')
Base.metadata.bind = engine
from sqlalchemy.orm import sessionmaker
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

def is_reviewed(s):
    return s.review1_timestamp>-1 and s.review2_timestamp>-1

def is_complete(s):
    if is_reviewed(s):
        a=s.assignment
        p=s.problem
        r1 = s.reviewer1
        r2 = s.reviewer2
        subs = session.query(Submission).filter( and_(or_(Submission.netid==r1,Submission.netid==r2),Submission.assignment ==a,Submission.problem ==p))
        if is_reviewed(subs[0]) and is_reviewed(subs[1]):
            return True
        else: 
            return False
    else:
        return False
    
def is_matched(s):
    if s.reviewer1_assignment_time>-1 and s.reviewer2_assignment_time>-1:
        return True
    else:
        return False
    
def get_grade(user,ass):
    """
    get_grade('yshi9',1)
    7.875473484848484
    """
    subs=session.query(Submission).filter( and_(Submission.netid==user, Submission.assignment==ass))
    subs = sorted(subs, key=lambda s: s.total_score2)
    return np.mean([s.total_score2 for s in subs[0:2]])

def get_email(netid):
    user=session.query(User).filter(User.netid==netid).first()
    return user.email

def strdate_to_int(s):
    time.mktime(datetime.datetime.strptime(s, "%m/%d/%Y").timetuple())
    
Base.metadata.create_all(engine)

In [5]:
# Insert an Address in the address table
algebra = Course(
    coursename = 'algebra-one',
    year=20,
    term = 'f', #F=Fall, S=Spring, U=Summer,
    coursenumber = 'Math 251', #used to be class
    uvm_course = 1,
    zulipurl = 'https://uvm-mathematics.zulipchat.com',
    zuliprc = '',
    homepage = 'https://www.uvm.edu/~tdupuy/',
    size = 36
)
session.add(algebra)
session.commit()

In [6]:
# Insert an Address in the address table
algebraic_topology = Course(
    coursename = 'algebraic-topology',
    year=20,
    term = 'f', #F=Fall, S=Spring, U=Summer,
    coursenumber = 'Math 354', #used to be class
    uvm_course = 1,
    zulipurl = 'https://uvm-mathematics.zulipchat.com',
    zuliprc = '',
    homepage = 'https://www.uvm.edu/~tdupuy/',
    size = 9
)
session.add(algebraic_topology)
session.commit()

"""
from sqlalchemy.ext.declarative import DeclarativeMeta

class AlchemyEncoder(json.JSONEncoder):

    def default(self, obj):
        if isinstance(obj.__class__, DeclarativeMeta):
            # an SQLAlchemy class
            fields = {}
            for field in [x for x in dir(obj) if not x.startswith('_') and x != 'metadata']:
                data = obj.__getattribute__(field)
                try:
                    json.dumps(data) # this will fail on non-encodable values, like other classes
                    fields[field] = data
                except TypeError:
                    fields[field] = None
            # a json-encodable dict
            return fields

        return json.JSONEncoder.default(self, obj)
    """

"\nfrom sqlalchemy.ext.declarative import DeclarativeMeta\n\nclass AlchemyEncoder(json.JSONEncoder):\n\n    def default(self, obj):\n        if isinstance(obj.__class__, DeclarativeMeta):\n            # an SQLAlchemy class\n            fields = {}\n            for field in [x for x in dir(obj) if not x.startswith('_') and x != 'metadata']:\n                data = obj.__getattribute__(field)\n                try:\n                    json.dumps(data) # this will fail on non-encodable values, like other classes\n                    fields[field] = data\n                except TypeError:\n                    fields[field] = None\n            # a json-encodable dict\n            return fields\n\n        return json.JSONEncoder.default(self, obj)\n    "

In [7]:
algebra=session.query(Course).filter(and_(
    Course.coursename == 'algebra-one',
    Course.year==2020,
    Course.term == 'f')
).first()

topology=session.query(Course).filter(and_(
    Course.coursename == 'algebraic-topology',
    Course.year==20,
    Course.term == 'f')
).first()

In [8]:
algebra=session.query(Course).all()[0]
print(algebra)

In [9]:
myproblems = A.get({})

for x in myproblems:
    new_problem = Problem(
    coursename = 'algebra-one', #algebra-one, algebraic-topology, agittoc1, agittoc2, agittoc3
    year = 2020,
    term = 'f', #f=Fall, s=Spring, u=Summer
    assignment = x['assignment'],
    problem = x['problem'],
    #references = x['']
    datasets={'subs':[],
                  'waiting':[],
                  'matched':[],
                  'reviewed':[],
                  'completed':[],
                  'scores':[],
                  'rt':[],
                  'mt':[]}
    )
    session.add(new_problem)

session.commit()

In [10]:
print(algebra)

In [11]:
date_str = '12/29/2017' # The date - 29 Dec 2017
format_str = '%m/%d/%Y' # The format
datetime_obj = datetime.datetime.strptime(date_str, format_str)
datetime_obj.timestamp()

1514523600.0

In [12]:
def string_to_timestamp(date_str, format_str='%m/%d/%Y'):
    datetime_obj = datetime.datetime.strptime(date_str, format_str)
    return datetime_obj.timestamp()

In [13]:
for p in session.query(Problem).all():
    #print(p)
    results=P.get({"assignment":int(p.assignment),"problem":int(p.problem)})
    #print(results)
    if len(results)==1:
        #print(results[0])
        result = results[0]
        p.description=result['problem_details']
        p.hints=result['hints']
        p.due_date=result['due_date'].timestamp()
        #print(p.due_date)
        p.course=algebra
        #print(p.course) this seems to be storing things properly
session.commit()

In [14]:
algebra2=session.query(Course).all()[0] #this returned something nontrivial
#print(algebra2.problems)
myproblem = session.query(Problem).all()[3]
print(myproblem)
print(myproblem.hints) #this should be None most of the time
print(myproblem.due_date) #this is working now, the zeroeth problem doesn't work
print(myproblem.course)

None
1599796800


In [15]:
print(myproblem)
myproblem.course=algebra
print(myproblem.course)
print(algebra.problems) #this is outputting stuff now. Before it seems we were not recording the problems correctly.

[<database_declarative.Problem object at 0x1264b1710>, <database_declarative.Problem object at 0x1264b1780>, <database_declarative.Problem object at 0x1266da438>, <database_declarative.Problem object at 0x1264b1f28>, <database_declarative.Problem object at 0x1264b1128>, <database_declarative.Problem object at 0x1264b14a8>, <database_declarative.Problem object at 0x1264b1048>, <database_declarative.Problem object at 0x1264b18d0>, <database_declarative.Problem object at 0x1264ba048>, <database_declarative.Problem object at 0x1264baf98>, <database_declarative.Problem object at 0x1264ba978>, <database_declarative.Problem object at 0x1264ba588>, <database_declarative.Problem object at 0x1264bad30>, <database_declarative.Problem object at 0x1264ba320>, <database_declarative.Problem object at 0x1264baac8>, <database_declarative.Problem object at 0x1264ba128>, <database_declarative.Problem object at 0x1263fbf28>, <database_declarative.Problem object at 0x1263fbd30>, <database_declarative.Probl

In [16]:
#session.rollback()
myusers = R.get({})

# Insert a Person in the person table
for x in myusers:
    new_person = User(
        name = x['name'],
        pronoun=x['pronoun'],
        uvm_student=1,
        student_id = x['student_id'],
        courses = 'algebra-one/f/20',
        major = x['major'],
        degree = x['degree'],
        credits = x['credits'],
        reg_status = x['reg_status'],
        reg_data = x['reg_data'],
        email = x['email'].lower(),
        netid = x['netid'],
        password = generate_password_hash(x['password'],method='sha256'),
        config={'courses':[{'id':algebra.id,'group':'user'}],'emails':[],'super_user':0})
    session.add(new_person)

session.commit()

In [17]:
mysubs = S.get({})
for x in mysubs:
    new_sub = Submission(
        coursename = 'algebra-one',
        year = '2020',
        term = 'f',
        submission_number = x['submission_number'],
        submission_locked = x['submission_locked'],
        netid = x['netid'],
        email = get_email(x['netid']),
        assignment = x['assignment'],
        problem = x['problem'],
        closed = x['closed'],
        submission_time = x['submission_time'],
        total_score1 = x['total_score1'],
        total_score2 = x['total_score2'],
        reviewer1_assignment_time =x['reviewer1_assignment_time'],
        reviewer1 = x['reviewer1'],
        reviewer1_email = '',
        reviewer1_score = x['reviewer1_score'],
        review1 = x['review1'],
        review1_timestamp = x['review1_timestamp'],
        review1_locked = x['review1_locked'],
        #bad1, bad2
        reviewer2_assignment_time =x['reviewer2_assignment_time'],
        reviewer2 = x['reviewer2'],
        reviewer2_email = '',
        reviewer2_score = x['reviewer2_score'],
        review2 = x['review2'],
        review2_timestamp = x['review2_timestamp'],
        review2_locked = x['review2_locked'],
        new_submission = x['new_submission'],
        new_match = x['new_match'],
        new_review1 = x['new_review1'],
        new_review2 = x['new_review2'],
        new_completion = x['new_completion'])
    session.add(new_sub)

session.commit()

In [18]:
#need to correct the submissions
for s in session.query(Submission).all():
    prob=session.query(Problem).filter(and_(
        Problem.coursename==s.coursename,
        Problem.problem==s.problem,
        Problem.assignment==s.assignment)).first()
    course=session.query(Course).filter(
        Course.coursename==s.coursename).first()
    user=session.query(User).filter(User.netid==s.netid).first()
    s.prob=prob
    s.course=course
    s.user=user
session.commit()

In [19]:
#this needs to be fixed for AGITTOC
student = 'tfbloom'
myuser = session.query(User).filter(User.netid==student).first()
email=myuser.email
subs=session.query(Submission).filter(Submission.email==email)
email

'tyler.bloom@uvm.edu'

In [79]:
print(myuser.config['super_user'])
print(myuser.submissions) #
print(myproblem.submissions) #

0
[<database_declarative.Submission object at 0x11fe6e278>, <database_declarative.Submission object at 0x11fe6e390>, <database_declarative.Submission object at 0x11fe6e668>, <database_declarative.Submission object at 0x11fe6eac8>, <database_declarative.Submission object at 0x11fe6ecc0>, <database_declarative.Submission object at 0x11fe6edd8>, <database_declarative.Submission object at 0x11fe6ebe0>, <database_declarative.Submission object at 0x11fe6eba8>, <database_declarative.Submission object at 0x11fe6e400>, <database_declarative.Submission object at 0x11fe6eb70>, <database_declarative.Submission object at 0x11fe6ecf8>, <database_declarative.Submission object at 0x11fe6e9e8>]


In [9]:
reviewsubs = session.query(Submission).filter( or_(Submission.reviewer1_email==email,Submission.reviewer2_email==email)).all()
reviewsubs

In [25]:
fresh_subs=session.query(Submission).filter(and_(Submission.netid==student, Submission.new_submission==1) ).all()
matched_subs=session.query(Submission).filter(and_(Submission.netid==student, Submission.submission_locked==1)).all()

first_reviews = []
for s in matched_subs:
    if is_reviewed(s):
        first_reviews.append(s)

for c in first_reviews:
    matched_subs.remove(c)        
        
completed =[]
for s in first_reviews:
    if is_complete(s):
        completed.append(s)
        
for c in completed:
    first_reviews.remove(c)

fresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])
matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
first_reviews = sorted(first_reviews,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])

In [19]:
for s in fresh_subs:
    print(f"{s.submission_number} (waiting): assignment {s.assignment}, problem {s.problem}")
    
for s in matched_subs:
    print(f"{s.submission_number} (matched): assignment {s.assignment}, problem {s.problem}")
    
for s in first_reviews:
    print(f"{s.submission_number} (first review done): assignment {s.assignment}, problem {s.problem}")
    
for s in completed:
    print(f"{s.submission_number} (complete): assignment {s.assignment}, problem {s.problem}")
    

NameError: name 'fresh_subs' is not defined

In [12]:
"""
completed =[]
for s in subs:
    if is_complete(s):
        completed.append(s)

fresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])
matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])
"""

'\ncompleted =[]\nfor s in subs:\n    if is_complete(s):\n        completed.append(s)\n\nfresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])\nmatched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])\ncompleted=sorted(completed,key=lambda c: [c.assignment,c.problem])\n'

In [13]:
for c in completed:
    print(f"{c.submission_number}: assignment {c.assignment}, problem {c.problem}, score {c.total_score2}")

In [14]:
closed_subs=[s.submission_number for s in subs.filter(Submission.closed==1)]

completed

In [15]:
s=subs[0]

IndexError: list index out of range

In [35]:
s.reviewer1
s.reviewer2

'cdecours'

In [36]:
is_reviewed(s)
a=s.assignment
p=s.problem
r1 = s.reviewer1
r2 = s.reviewer2
print(f'submission number: {s.submission_number}, author: {s.netid}, reviewer 1: {r1}, reviewer2: {r2}') 
reviews = session.query(Submission).filter( and_(or_(Submission.netid==r1,Submission.netid==r2),Submission.assignment ==a,Submission.problem ==p))
for rev in reviews:
    print(f" submission number: {rev.submission_number}; assignment {rev.assignment}, problem {rev.problem}, reviewer id {rev.netid}")

submission number: 92, author: yshi9, reviewer 1: ttamang, reviewer2: cdecours
 submission number: 70; assignment 1, problem 1, reviewer id cdecours
 submission number: 79; assignment 1, problem 1, reviewer id ttamang


In [45]:
is_complete(s)

False

In [ ]:
s.review1_timestamp

In [37]:
def poke_reviewers(sub_num):
    subject = "re"
    sub=session.query(Submission).filter(Submission.submission_number=sub_num)
    if s.review1_timestamp==-1:
        myemail['sender'] == 
        out_msg=send_email(myemail)
    if s.review2_timestamp==-1:        

SyntaxError: invalid syntax (<ipython-input-37-db720c296e9f>, line 5)

In [ ]:
    emails[user]['subject'] = course_name + " Database Updates " + ("%s" % datetime.datetime.now())
    #emails[user]['message_parts'] = {}
    emails[user]['message'] =''
    emails[user]['attachments'] = []
    emails[user]['receiver'] = R.get({'netid':user})[0]['email']

In [17]:
matched_subs = session.query(Submission).filter( or_(Submission.netid==student,Submission.netid==student)).all()

In [18]:
first_reviews = []
for s in matched_subs:
    if is_reviewed(s):
        first_reviews.append(s)

for c in first_reviews:
    matched_subs.remove(c)        
        
completed =[]
for s in first_reviews:
    if is_complete(s):
        completed.append(s)
        
for c in completed:
    first_reviews.remove(c)

matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
first_reviews = sorted(first_reviews,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])

In [16]:
session.query(Course).all()[0].coursename

'algebra-one'

In [16]:
mydict={"a":1,"b":2,"c":3}
list(mydict.keys())

['a', 'b', 'c']

In [11]:
x=None

In [12]:
x==None

True

In [13]:
y=1
y==None

False